### OpenAI 1113 api update 

In [70]:
import os,sys
sys.path.insert(0,'../libs')
import time
from utils import load_json,logging,exception_handler
from llm_utils import get_oai_fees
import json
## load API Key

key = load_json('/root/workspace/Dev/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT1']['API_KEY']
import openai
from openai import OpenAI

In [2]:
## define a base openai agent class 
class BSAgent():
    def __init__(self, api_key=None, 
                 model="gpt-3.5-turbo-1106", 
                 temperature=0):

        if not api_key:
            api_key = os.environ['OPENAI_API_KEY']
        self.client = OpenAI(api_key=api_key)
        self.temperature = temperature
        if model:
            self.model = model
        self.message = []

    def _add_messages_history(self, response) -> None:

        self.message.append(response["choices"][0]["message"])

    @exception_handler(error_msg='Failed with multiple retry',
                        error_return=None,
                        attempts=3,delay=5)
    def get_completion(self,
                       prompt_template, 
                       model=None,
                       temperature=None,
                       conv_history=[],
                       return_cost=False,
                       verbose=True,
                       stream=False):
        if not model:
            model = self.model
        
        if not temperature:
            temperature = self.temperature
        
        new_message = []
        if prompt_template.get('System'):
            new_message.append({"role": "system", "content": prompt_template['System']})
        if prompt_template.get('Human'):
            new_message.append({"role": "user", "content": prompt_template['Human']})
        
        conv_history.extend(new_message)
        
        if len(conv_history) == 0 :
            raise Exception('prompt template error, prompt must be a dict with with System message or Human message.')
        
        response = self.client.chat.completions.create(
                    model=model,
                    messages=conv_history,
                    temperature=temperature,
                    stream=stream
                )
        
        if not stream:
            prompt_tokens = response.usage.prompt_tokens
            completion_tokens = response.usage.completion_tokens
            this_time_price = get_oai_fees(model, prompt_tokens, completion_tokens)
            if verbose:
                logging.info(f"************|||----|||price: {this_time_price}|||----|||************")
        
        if return_cost:
            return response,this_time_price
        
        return response 
    
    def get_response_content(self,**kwargs):
        response = self.get_completion(**kwargs)
        res_msg = response.choices[0].message.content
        return res_msg


In [3]:
## initiate an llm agent
llm_agent = BSAgent(model="gpt-3.5-turbo-1106")

In [4]:
pt = {'System':'You are a helpful assistant.',
      'Human':'What is your name?'}
res = llm_agent.get_response_content(prompt_template=pt)
print(res)

I'm an AI language model and don't have a personal name. You can just call me Assistant. How can I assist you today?


### Use Stream

In [18]:
completion = llm_agent.get_completion(prompt_template=pt,stream=True)
for chunk in completion:
  print(chunk.choices[0].delta.content,end="")

I'm a helpful assistant here to answer your questions and assist you with tasks. You can call me "Assistant."None

### Batch complete
- https://platform.openai.com/docs/guides/rate-limits/batching-requests

### GPT4 vision preview

In [5]:
response = llm_agent.client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason=None, index=0, message=ChatCompletionMessage(content='The image shows a scenic natural landscape featuring a wooden boardwalk extending across a green grassy field. The boardwalk leads towards the horizon and seems to provide a path through the high grass, perhaps in a wetland or marsh area. The sky is partly cloudy with fluffy blue and white clouds, suggesting it is either a late afternoon or a tranquil morning. The greenery and the clear sky contribute to the serenity of the landscape. There are also some shrubs or bushes and a few trees scattered in the distance, accentuating the diverse vegetation of the area. The image exudes a sense of peace and might be used to depict topics related to nature walks, conservation areas, wetlands, or the beauty of untouched landscapes.', role='assistant', function_call=None, tool_calls=None), finish_details={'type': 'stop', 'stop': '<|fim_suffix|>'})


### OpenAI Assistnat API Demo
- ### Assistant API for retrieval

In [28]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [29]:
## upload a file 
sample_file = '/home/chengyu.huang/project/Fund_projects/temp_data/174_638_R0.pdf'
assert os.path.exists(sample_file)
file = client.files.create(
    file=open(sample_file,'rb'),
    purpose='assistants'
)
print(file.id)

file-0IovNjuAFjxEXjkIqrQ6Vga6


In [30]:
# list out existing files 
file_list = client.files.list()
print(file_list)

SyncPage[FileObject](data=[FileObject(id='file-0IovNjuAFjxEXjkIqrQ6Vga6', bytes=1726558, created_at=1699990845, filename='174_638_R0.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False)


- Create the assistant, give it access to the file

In [31]:
# Add the file to the assistant 
file_id = file_list.data[0].id
assistant = client.beta.assistants.create(
    instructions="You are a chatbot designed to respond to enquires about the IMF lending program. Use your knowledge base to best respond to user questions",
    model="gpt-3.5-turbo-1106",
    tools=[{"type":"retrieval"}],
    file_ids = [file_id]
)
## you can use client.beta.assistants.update if you need to change some of the settings
print(assistant.id)

asst_S7tADW8enJ07mcZt11hSOgzx


- You can also retrieve an existing assistant

In [32]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20"
)
print(my_assistants.data)

[Assistant(id='asst_S7tADW8enJ07mcZt11hSOgzx', created_at=1699990853, description=None, file_ids=['file-0IovNjuAFjxEXjkIqrQ6Vga6'], instructions='You are a chatbot designed to respond to enquires about the IMF lending program. Use your knowledge base to best respond to user questions', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])]


In [38]:
## retrieive a assistant 
assistant_id = my_assistants.data[0].id
assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)


Assistant(id='asst_S7tADW8enJ07mcZt11hSOgzx', created_at=1699990853, description=None, file_ids=['file-0IovNjuAFjxEXjkIqrQ6Vga6'], instructions='You are a chatbot designed to respond to enquires about the IMF lending program. Use your knowledge base to best respond to user questions', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])


- Create an empty thread and add message to the thread and run the assistant 

In [39]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_eWfRuMQm2RFhMT4DrciYFzO8', created_at=1699990941, metadata={}, object='thread')


In [93]:
## create message and add to the thread 
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you list the conditionalities required in the imf program"
)

In [94]:
## run the assistant 
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions="Please address the user as Huang. Do not provide an answer if no usefule infomation was retrieved from konwledge base."
)

print(run.id)


run_SorRFLhD2CGH4e4sfOFBFGBn


In [95]:
## retrieve the result 
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run.status)

in_progress


In [63]:
## list out run steps 
# run_steps = client.beta.threads.runs.steps.list(
#     thread_id = thread.id,
#     run_id = run.id
# )
# print(run_steps)

SyncCursorPage[RunStep](data=[RunStep(id='step_xxYp8F1qsSr5kYwZc8FXX79N', assistant_id='asst_S7tADW8enJ07mcZt11hSOgzx', cancelled_at=None, completed_at=1699990947, created_at=1699990944, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_iX4jsjgKM7arTXqiIOmKw1Tu', status='completed', step_details=MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_jk1C9PXMX7gUl2zXXjgGp3qj'), type='message_creation'), thread_id='thread_eWfRuMQm2RFhMT4DrciYFzO8', type='message_creation', expires_at=None), RunStep(id='step_EwMQMwdvEeutVglYS6O7eyEK', assistant_id='asst_S7tADW8enJ07mcZt11hSOgzx', cancelled_at=None, completed_at=1699990944, created_at=1699990943, expired_at=None, failed_at=None, last_error=None, metadata=None, object='thread.run.step', run_id='run_iX4jsjgKM7arTXqiIOmKw1Tu', status='completed', step_details=ToolCallsStepDetails(tool_calls=[RetrievalToolCall(id='call_OFhhc0AJ2DKPloJ3HRqwVZx3', retrieval={}, type='retri

In [70]:
### how to cancle a run 
# run = client.beta.threads.runs.cancel(
#     thread_id=thread.id,
#     run_id = run.id
# )

In [103]:
## retrieve the message 
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for each in messages:
  print(each.role+": {}".format(each.content[0].text.value))
  print("=============")
##print(messages.data[0].content[0].text.value)

assistant: It seems that the specific list of conditionalities is not explicitly stated in the document. If there are any specific terms, policy measures, or requirements related to the IMF program that you would like me to look for, please let me know and I can conduct a search for those specific details.
assistant: I'm unable to retrieve the specific conditionalities from the document using the quote function. Let me try another approach to locate the details.
user: Can you list the conditionalities required in the imf program
assistant: The document provides a summary of the conditionalities related to the request for a Stand-By Arrangement for Greece. Here are the main elements of the program:

1. Greece is adopting an ambitious comprehensive multi-year adjustment program to lower the fiscal deficit and the debt ratio, reduce domestic demand in line with capacity, and increase supply and competitiveness.
2. The requested Stand-By Arrangement is for €30 billion with an initial purch

- Post test clean up 

In [105]:
my_assistants = client.beta.assistants.list(
    order='desc',
    limit="20"
)
print(my_assistants.data)

[Assistant(id='asst_S7tADW8enJ07mcZt11hSOgzx', created_at=1699990853, description=None, file_ids=['file-0IovNjuAFjxEXjkIqrQ6Vga6'], instructions='You are a chatbot designed to respond to enquires about the IMF lending program. Use your knowledge base to best respond to user questions', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])]


In [106]:
response = client.beta.assistants.delete('asst_S7tADW8enJ07mcZt11hSOgzx')
print(response)

AssistantDeleted(id='asst_S7tADW8enJ07mcZt11hSOgzx', deleted=True, object='assistant.deleted')


- ### Assistnat API, function call
- give the assistant additional tools 
- example : https://medium.com/@assafelovic/how-to-build-an-openai-assistant-with-internet-browsing-ee5ad7625661

In [6]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [13]:
# list out existing files , get the firt file id 
file_list = client.files.list()
print(file_list.data[0].filename)
file_id = file_list.data[0].id
print(file_id)

174_638_R0.pdf
file-0IovNjuAFjxEXjkIqrQ6Vga6


- define some customized functions
- Here is an example of a web search 

In [101]:
## some demo function descriptions to register in openai
web_search ={
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Get information on recent events from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query to use. For example: 'Latest news on Nvidia stock performance'"},
                },
                "required": ["query"]
            }
        }
    }

get_whether={
      "type": "function",
    "function": {
      "name": "getCurrentWeather",
      "description": "Get the weather in location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {"type": "string", "description": "The city and state e.g. San Francisco, CA"},
          "unit": {"type": "string", "enum": ["c", "f"]}
        },
        "required": ["location"]
      }
    }
  }

assistant_prompt_instruction = """You are an expert on IMF programs. 
Your goal is to provide answers based on information from the internet and the knowledge base. 
You must use the provided web search function to find relevant online information. 
On top of web info, you should also use your own knowledge to answer questions.
Please include relevant url sources in the end of your answers.
"""

- create an assistant

In [102]:
assistant = client.beta.assistants.create(
    instructions=assistant_prompt_instruction,
    model="gpt-4-1106-preview",
    tools=[{"type":"retrieval"},
           web_search,
           ],
    file_ids=[file_id]
)
print(assistant.id)
# response = client.beta.assistants.delete(assistant.id)
# print(response)
# you can use client.beta.assistants.update
# or client.beta.assistants.retrieve
# for reuse existing assistant

asst_R2EUD7DAltjLs8CNCHK2Nbz1


- create a thread and run 

In [ ]:
thread = client.beta.threads.create()

In [127]:
user_input = 'what is the most recent developments on the program?' #input("You: ")

In [128]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_input,
)

In [129]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
print(run.id)


run_1aX1XfBfTzrGqMwrQJ3pJDIG


In [130]:
# Function to wait for a run to complete
def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

run = wait_for_run_completion(thread.id,run.id)


Current run status: requires_action


In [131]:
tools_to_call = run.required_action.submit_tool_outputs.tool_calls
print(tools_to_call)
print('function name: {}'.format(tools_to_call[0].function.name))

[RequiredActionFunctionToolCall(id='call_FJ8YfNR7IzXlG30zFop0yPGZ', function=Function(arguments='{"query":"latest IMF program updates April 2023"}', name='web_search'), type='function')]
function name: web_search


In [132]:
def web_search_func(q):
    """
    a dummy websearch function for testing purpose
    """
    res = "there isn't much new developments on the program."
    return res

In [133]:
# Function to handle tool output submission
def submit_tool_outputs(tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "web_search":
            output = web_search_func(q=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    return tool_output_array

In [134]:
tool_res = submit_tool_outputs(tools_to_call)
print(tool_res)
    

[{'tool_call_id': 'call_FJ8YfNR7IzXlG30zFop0yPGZ', 'output': "there isn't much new developments on the program."}]


In [135]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_res
)

In [136]:
run = wait_for_run_completion(thread.id,run.id)

Current run status: in_progress
Current run status: in_progress
Current run status: completed


- see the messages, looks like search results are incoporated in answers

In [138]:
## retrieve the message 
return_messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
for each in return_messages:
  print(each.role+": {}".format(each.content[0].text.value))
  print("=============")
##print(messages.data[0].content[0].text.value)

assistant: It appears that there are no significant new developments on IMF programs as of the latest information available up to April 2023. The IMF often has several ongoing programs with member countries, and the status or updates of these can vary significantly based on the country and the specific circumstances of the program. If there is a particular country or program you are interested in, please provide more details so that I can search for the most recent developments related to that specific case.
user: what is the most recent developments on the program?
assistant: The most recent general developments on the International Monetary Fund (IMF) programs include the IMF's continued effort to provide financial support to countries affected by economic crises, giving them breathing space to implement policies aimed at restoring economic stability and growth. Additionally, the IMF provides precautionary financing to help prevent crises. IMF lending practices are consistently revis